# Time Series Data Preparation

Walmart Sales dataset, collected from [Kaggle](https://www.kaggle.com/c/walmart-recruiting-store-sales-forecasting/data), contains historical sales data for 45 Walmart stores in the United State from 2/5/2010 to 11/1/2012.

In [1]:
import module.util_functions as utf
import module.constants as const
import zipfile
import datetime
import pandas as pd
import numpy as np
import os

# load stores data
stores = pd.read_csv(os.path.join(const.DATA_DIR, 'stores.csv'))

# create store type dictionary
store_type_dict = stores.set_index(stores.Store)['Type'].to_dict()

# load sales data
sales = pd.read_csv(os.path.join(const.DATA_DIR, 'train.csv.zip'),compression='zip')

# convert data type for Date from object to datetime
sales['Date'] = pd.to_datetime(sales['Date'])
sales['Type'] = sales['Store'].map(store_type_dict)     # set store's type

In [2]:
sales.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Type
0,1,1,2010-02-05,24924.50,False,A
1,1,1,2010-02-12,46039.49,True,A
2,1,1,2010-02-19,41595.55,False,A
3,1,1,2010-02-26,19403.54,False,A
4,1,1,2010-03-05,21827.90,False,A


In [3]:
# get a list of unique dates sorted in ascending order
dates = sales['Date'].sort_values().unique()

# set a time period for each date
# earliest date has a time period of 1
# successive dates has a time period of 2, 3, ..., 143
dates_dict = dict(zip(dates, np.arange(1, len(dates) + 1)))

## Prepare Time Series Dataset

### Compute Weekly Sales per Store

In [4]:
# sum department sales per store for each Date
# then set store's type
store_sales = sales.groupby(['Store', 'Date'])['Weekly_Sales'].sum().reset_index()
store_sales['Type'] = store_sales['Store'].map(store_type_dict)

# extract week number of year from Date
# then scale Week to have a range from 0 to 1, and save under new name
store_sales['Week'] = store_sales['Date'].map(lambda x: datetime.date(x.year, x.month, x.day).isocalendar().week)
store_sales['Scaled_Week'] = store_sales['Week'].apply(lambda x: x / 52)
store_sales.head()

,Store,Date,Weekly_Sales,Type,Week,Scaled_Week
0,1,2010-02-05,1643690.90,A,5,0.096154
1,1,2010-02-12,1641957.44,A,6,0.115385
2,1,2010-02-19,1611968.17,A,7,0.134615
3,1,2010-02-26,1409727.59,A,8,0.153846
4,1,2010-03-05,1554806.68,A,9,0.173077


### Scale Sale Values

In [5]:
# scale sales value down by million
store_sales['Weekly Sales (Million)'] = store_sales['Weekly_Sales'] / 1000000

# drop Weekly_Sales
del store_sales['Weekly_Sales']
store_sales.head()

,Store,Date,Type,Week,Scaled_Week,Weekly Sales (Million)
0,1,2010-02-05,A,5,0.096154,1.643691
1,1,2010-02-12,A,6,0.115385,1.641957
2,1,2010-02-19,A,7,0.134615,1.611968
3,1,2010-02-26,A,8,0.153846,1.409728
4,1,2010-03-05,A,9,0.173077,1.554807


In [6]:
store_sales['Weekly Sales (Million)'].describe()

count    6435.000000
mean        1.046965
std         0.564367
min         0.209986
25%         0.553350
50%         0.960746
75%         1.420159
max         3.818686
Name: Weekly Sales (Million), dtype: float64

### Assign Holiday Name

In [7]:
# get holiday's name if holiday falls within any specific weeks
store_sales['Holiday'] = store_sales['Date'].apply(utf.get_holiday_name)

In [8]:
store_sales[store_sales.Holiday != ''].head()

,Store,Date,Type,Week,Scaled_Week,Weekly Sales (Million),Holiday
1,1,2010-02-12,A,6,0.115385,1.641957,Super Bowl
31,1,2010-09-10,A,36,0.692308,1.507461,Labor Day
42,1,2010-11-26,A,47,0.903846,1.955624,Thanksgiving
47,1,2010-12-31,A,52,1.000000,1.367320,Christmas
53,1,2011-02-11,A,6,0.115385,1.649615,Super Bowl


In [9]:
# create dummy variables for each holiday
for holiday_name in ['Super Bowl', 'Labor Day', 'Thanksgiving', 'Before Christmas', 'Christmas']:
    if holiday_name != 'Before Christmas':
        store_sales[holiday_name] = store_sales['Holiday'].apply(lambda x: 1 if x == holiday_name else 0)
    else:  # Before Christmas
        store_sales[holiday_name] = store_sales['Week'].apply(lambda x: 1 if x == 51 else 0)

### Indexing Time Series

In [10]:
# set time period based on Date
store_sales['Time Period'] = store_sales['Date'].map(dates_dict)

In [11]:
store_sales.tail()

,Store,Date,Type,Week,Scaled_Week,Weekly Sales (Million),Holiday,Super Bowl,Labor Day,Thanksgiving,Before Christmas,Christmas,Time Period
6430,45,2012-09-28,B,39,0.750000,0.713174,,0,0,0,0,0,139
6431,45,2012-10-05,B,40,0.769231,0.733455,,0,0,0,0,0,140
6432,45,2012-10-12,B,41,0.788462,0.734464,,0,0,0,0,0,141
6433,45,2012-10-19,B,42,0.807692,0.718126,,0,0,0,0,0,142
6434,45,2012-10-26,B,43,0.826923,0.760281,,0,0,0,0,0,143


## Merge Store's Sales and Features Dataframes

In [12]:
# load features
features = pd.read_csv('data/features.zip', compression='zip')
features['Date'] = pd.to_datetime(features['Date'])
features.index = pd.MultiIndex.from_frame(features[['Store', 'Date']])
features.head()

Store       Date  Temperature  Fuel_Price  MarkDown1  \
Store Date                                                               
1     2010-02-05      1 2010-02-05        42.31       2.572        NaN   
      2010-02-12      1 2010-02-12        38.51       2.548        NaN   
      2010-02-19      1 2010-02-19        39.93       2.514        NaN   
      2010-02-26      1 2010-02-26        46.63       2.561        NaN   
      2010-03-05      1 2010-03-05        46.50       2.625        NaN   

                  MarkDown2  MarkDown3  MarkDown4  MarkDown5         CPI  \
Store Date                                                                 
1     2010-02-05        NaN        NaN        NaN        NaN  211.096358   
      2010-02-12        NaN        NaN        NaN        NaN  211.242170   
      2010-02-19        NaN        NaN        NaN        NaN  211.289143   
      2010-02-26        NaN        NaN        NaN        NaN  211.319643   
      2010-03-05        NaN        NaN        NaN        NaN  211.350143   

                  Unemployment  IsHoliday  
Store Date                                 
1     2010-02-05         8.106      False  
      2010-02-12         8.106       True  
      2010-02-19         8.106      False  
      2010-02-26         8.106      False  
      2010-03-05         8.106      False

In [13]:
features.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 8190 entries, (1, Timestamp('2010-02-05 00:00:00')) to (45, Timestamp('2013-07-26 00:00:00'))
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   Store         8190 non-null   int64         
 1   Date          8190 non-null   datetime64[ns]
 2   Temperature   8190 non-null   float64       
 3   Fuel_Price    8190 non-null   float64       
 4   MarkDown1     4032 non-null   float64       
 5   MarkDown2     2921 non-null   float64       
 6   MarkDown3     3613 non-null   float64       
 7   MarkDown4     3464 non-null   float64       
 8   MarkDown5     4050 non-null   float64       
 9   CPI           7605 non-null   float64       
 10  Unemployment  7605 non-null   float64       
 11  IsHoliday     8190 non-null   bool          
dtypes: bool(1), datetime64[ns](1), float64(9), int64(1)
memory usage: 742.8 KB


In [14]:
# set multi-index
store_sales.index = pd.MultiIndex.from_frame(store_sales[['Store', 'Date']])
store_sales.head()

Store       Date Type  Week  Scaled_Week  \
Store Date                                                   
1     2010-02-05      1 2010-02-05    A     5     0.096154   
      2010-02-12      1 2010-02-12    A     6     0.115385   
      2010-02-19      1 2010-02-19    A     7     0.134615   
      2010-02-26      1 2010-02-26    A     8     0.153846   
      2010-03-05      1 2010-03-05    A     9     0.173077   

                  Weekly Sales (Million)     Holiday  Super Bowl  Labor Day  \
Store Date                                                                    
1     2010-02-05                1.643691                       0          0   
      2010-02-12                1.641957  Super Bowl           1          0   
      2010-02-19                1.611968                       0          0   
      2010-02-26                1.409728                       0          0   
      2010-03-05                1.554807                       0          0   

                  Thanksgiving  Before Christmas  Christmas  Time Period  
Store Date                                                                
1     2010-02-05             0                 0          0            1  
      2010-02-12             0                 0          0            2  
      2010-02-19             0                 0          0            3  
      2010-02-26             0                 0          0            4  
      2010-03-05             0                 0          0            5

In [15]:
store_sales.index = pd.MultiIndex.from_frame(store_sales[['Store', 'Date']])
store_sales.head()

Store       Date Type  Week  Scaled_Week  \
Store Date                                                   
1     2010-02-05      1 2010-02-05    A     5     0.096154   
      2010-02-12      1 2010-02-12    A     6     0.115385   
      2010-02-19      1 2010-02-19    A     7     0.134615   
      2010-02-26      1 2010-02-26    A     8     0.153846   
      2010-03-05      1 2010-03-05    A     9     0.173077   

                  Weekly Sales (Million)     Holiday  Super Bowl  Labor Day  \
Store Date                                                                    
1     2010-02-05                1.643691                       0          0   
      2010-02-12                1.641957  Super Bowl           1          0   
      2010-02-19                1.611968                       0          0   
      2010-02-26                1.409728                       0          0   
      2010-03-05                1.554807                       0          0   

                  Thanksgiving  Before Christmas  Christmas  Time Period  
Store Date                                                                
1     2010-02-05             0                 0          0            1  
      2010-02-12             0                 0          0            2  
      2010-02-19             0                 0          0            3  
      2010-02-26             0                 0          0            4  
      2010-03-05             0                 0          0            5

In [16]:
# merge store_sales and features dataframes
store_sales = store_sales.join(features[['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 
                           'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']])
del store_sales['Store']
del store_sales['Date']
store_sales.reset_index(inplace=True)
store_sales.head()

,Store,Date,Type,Week,Scaled_Week,Weekly Sales (Million),Holiday,Super Bowl,Labor Day,Thanksgiving,...,Time Period,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment
0,1,2010-02-05,A,5,0.096154,1.643691,,0,0,0,...,1,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106
1,1,2010-02-12,A,6,0.115385,1.641957,Super Bowl,1,0,0,...,2,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106
2,1,2010-02-19,A,7,0.134615,1.611968,,0,0,0,...,3,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106
3,1,2010-02-26,A,8,0.153846,1.409728,,0,0,0,...,4,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106
4,1,2010-03-05,A,9,0.173077,1.554807,,0,0,0,...,5,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106


In [17]:
store_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Store                   6435 non-null   int64         
 1   Date                    6435 non-null   datetime64[ns]
 2   Type                    6435 non-null   object        
 3   Week                    6435 non-null   int64         
 4   Scaled_Week             6435 non-null   float64       
 5   Weekly Sales (Million)  6435 non-null   float64       
 6   Holiday                 6435 non-null   object        
 7   Super Bowl              6435 non-null   int64         
 8   Labor Day               6435 non-null   int64         
 9   Thanksgiving            6435 non-null   int64         
 10  Before Christmas        6435 non-null   int64         
 11  Christmas               6435 non-null   int64         
 12  Time Period             6435 non-null   int32   

## Log Transformation

In [18]:
# compute Log value of Weekly Sales (Million) for fitting ARIMA/SARIMA model
store_sales['Log of Weekly Sales (Million)'] = np.log(store_sales['Weekly Sales (Million)'])

## Weekly Sales Correlation per Store

In [19]:
store_corr = None    # correlation dataframe

# iterate through a list of stores
for store_number in sales['Store'].unique():
    # get weekly sales for a given store
    sales_by_store = store_sales[store_sales.Store == store_number]
    
    # compute correlation score per Weekly Sales (Million) and each features in the list
    corr = np.round(sales_by_store[['Weekly Sales (Million)', 'Temperature', 'Fuel_Price', 'CPI', 
                                    'Unemployment', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 
                                    'MarkDown5']].corr(), 2)
    corr = pd.DataFrame(corr.iloc[0]).T
    corr['Store'] = store_number
    corr['Type'] = store_sales['Type'].values[0]
    
    # concatenate correlation dataframes
    if store_corr is None:
        store_corr = corr
    else:
        store_corr = pd.concat([store_corr, corr], axis=0)


del store_corr['Weekly Sales (Million)']

## Save Preprocessed Data

In [20]:
# save dataset to csv files
store_sales.to_csv('data/store_sales.csv', index=False)
store_corr.to_csv('data/store_corr.csv', index=False)